# Installations & Util

In [ ]:
! pip install ipyfilechooser ipywidgets

## Code Hiding

In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML
# ========== TOGGLE BTN ==========
javascript_functions = {False: "hide()", True: "show()"}
button_descriptions  = {False: "Show code", True: "Hide code"}

def toggle_code(state):
    """
    Toggles the JavaScript show()/hide() function on the div.input element.
    """
    output_string = "<script>$(\"div.input\").{}</script>"
    output_args   = (javascript_functions[state],)
    output        = output_string.format(*output_args)
    display(HTML(output))

def button_action(value):
    """
    Calls the toggle_code function and updates the button description.
    """
    state = value.new
    toggle_code(state)
    value.owner.description = button_descriptions[state]
    
state = False
toggle_code(state)

button = widgets.ToggleButton(state, description = button_descriptions[state])
button.observe(button_action, "value")
# ========== END TOGGLE BTN END ==========
display(button)

# Data Exploration

### T10-4
As a USER, I want to have a Data Exploration Section in the notebook so that I can easily locate the upload and playback functions in the notebook.

# Inference

### T10-7
As a USER, I want to have an Inference Section in the notebook so that I can easily locate the Inference Section in the notebook.

### T10-24
Output video processing functions

In [ ]:
import cv2
import pandas as pd
from moviepy.editor import VideoFileClip

def output(frame):
    try:
        cv2.putText(frame, str(next(df)[1].captions), position, cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 3, cv2.LINE_AA)

    # Exception called when next in iterator ends so except error here
    except StopIteration:
        pass

    return frame


# Path to video
video_path = "video.mp4"

# Capture video
cap = cv2.VideoCapture(video_path)

# Get video height and width from captured video
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

# Set position for text with offset of text size
position = ((int((frame_width/2)-(268/2))), int(((frame_height/2)-(36/2))))

video = VideoFileClip(video_path)

# Path to caption
caption = "captions.csv"

df = pd.read_csv(caption).iterrows()

# Modifies the images of a clip by replacing the frame
out_video = video.fl_image(output)

# Write edited video to file
out_video.write_videofile("output.mp4", audio = True)

# Close video file clip
out_video.close()

### T10-6
As a USER, I want to have a Training Section in the notebook so that I can easily locate the training related functions in the notebook.

## File Upload

In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML
from ipyfilechooser import FileChooser
import zipfile as zf
from ipywidgets import Button

# ========== FileChooser ==========
# Create and display a FileChooser widget
fc = FileChooser('/Users')

# Change defaults and reset the dialog
# fc.default_path = './'
# fc.reset()

# Restrict navigation to /Users
fc.sandbox_path = '/Users'

# Change hidden files
fc.show_hidden = False

# Switch to folder-only mode
fc.show_only_dirs = False

# Set multiple file filter patterns (uses https://docs.python.org/3/library/fnmatch.html)
# fc.filter_pattern = ['*.jpg', '*.png','*.txt']
fc.filter_pattern = ['*.zip']

# Change the title (use '' to hide)
fc.title = '<b>Select File (*zip)</b>'

# Sample callback function
def change_display_selected(chooser):
    print(fc.selected_filename, end='\r')

# Register callback function
fc.register_callback(change_display_selected)

def on_button_clicked(b):
    # FILE UPLOAD
    print(fc.selected)
    fn = fc.selected_filename.replace('.zip', '')
    files = zf.ZipFile(fc.selected_path+"\\"+fc.selected_filename, 'r')
    files.extractall('Data Folder\\'+fn)
    files.close()
    with output:
        print("Files saved.")
# ========== END FileChooser END ==========
# ========== CONFIRM BTN ==========
confirmBtn = widgets.Button(
    description='Confirm Upload',
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Confirm',
)
output = widgets.Output()
confirmBtn.on_click(on_button_clicked)
# ========== CONFIRM BTN ==========
display(fc)
display(confirmBtn, output)

## Select File for Training

In [ ]:
from IPython import display
from ipyfilechooser import FileChooser
from IPython.display import display
from IPython.display import IFrame
import ipywidgets as widgets

output2 = widgets.Output()
# ========== FileChooser ==========
# Create and display a FileChooser widget
fc2 = FileChooser('./')
display(fc2)

# Restrict navigation to /Users
fc2.sandbox_path = './'

# Change hidden files
fc2.show_hidden = False

# Switch to folder-only mode
fc2.show_only_dirs = False

# Change the title (use '' to hide)
fc2.title = '<b>Select File for Training</b>'

# Sample callback function
def change_display_selected(chooser):
    with output2:
        print(fc2.selected_filename, end='\r')
    
# Register callback function
fc2.register_callback(change_display_selected)
# ========== END FileChooser END ==========
display(output2)